In [13]:
import pandas as pd

In [14]:
df = pd.read_csv('tracks.csv', skiprows=1, index_col=0, engine='python', error_bad_lines=False)

Skipping line 53396: unexpected end of data


In [15]:
tracks = df.copy()

In [16]:
tracks = tracks.dropna(subset=['genre_top'])

In [17]:
import category_encoders as ce

cat_encoder = ce.OrdinalEncoder(return_df=False)
tracks['genre_top_encoded'] = cat_encoder.fit_transform(tracks['genre_top'])

In [18]:
X = tracks.drop(columns=['genre_top', 'genre_top_encoded', 'id', 'id.1', 'genres', 'genres_all'], axis=1)
y = tracks['genre_top_encoded']
X.shape, y.shape

((28426, 47), (28426,))

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [20]:
X_train.columns

Index(['comments', 'date_created', 'date_released', 'engineer', 'favorites',
       'information', 'listens', 'producer', 'tags', 'title', 'tracks', 'type',
       'active_year_begin', 'active_year_end', 'associated_labels', 'bio',
       'comments.1', 'date_created.1', 'favorites.1', 'latitude', 'location',
       'longitude', 'members', 'name', 'related_projects', 'tags.1', 'website',
       'wikipedia_page', 'split', 'subset', 'bit_rate', 'comments.2',
       'composer', 'date_created.2', 'date_recorded', 'duration',
       'favorites.2', 'information.1', 'interest', 'language_code', 'license',
       'listens.1', 'lyricist', 'number', 'publisher', 'tags.2', 'title.1'],
      dtype='object')

In [21]:
X_train.shape

(22740, 47)

In [23]:
hash_features = ['producer', 'location', 'composer']
X_train_features = pd.DataFrame()
X_test_features = pd.DataFrame()
for hash_feature in hash_features:
    hasher = ce.HashingEncoder(n_components=10, cols=[hash_feature])
    hasher = hasher.fit(X_train[hash_feature])
    hasher_train = hasher.transform(X_train.loc[:,hash_feature])
    #hasher_train.columns = [str(col) + '_' + hash_feature for col in hasher_train.columns]
    hasher_test = hasher.transform(X_test.loc[:,hash_feature])
    #hasher_test.columns = [str(col) + '_' + hash_feature for col in hasher_test.columns]
    X_train_features = pd.concat([X_train_features, hasher_train], axis=1)
    X_test_features = pd.concat([X_test_features, hasher_test], axis=1)

In [24]:
X_train_features.shape, X_test_features.shape

((22740, 30), (5686, 30))

Dummy Encode the type columns

In [28]:
dummy_features = ['type']

for dummy_feature in dummy_features:
    dummy_encoder = ce.OneHotEncoder(cols =[dummy_feature])
    dummy_encoder = dummy_encoder.fit(X_train[dummy_feature])
    
    dummy_train = dummy_encoder.transform(X_train.loc[:,dummy_feature])
    dummy_test = dummy_encoder.transform(X_test.loc[:,dummy_feature])
    
    X_train_features = pd.concat([X_train_features, dummy_train], axis=1)
    X_test_features = pd.concat([X_test_features, dummy_test], axis=1)

Test with Category Transformed Features

In [36]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='lbfgs', max_iter=1000, multi_class='auto')
log_reg = log_reg.fit(X_train_features, y_train)

score = log_reg.score(X_train_features, y_train)
print('Train Accuracy Score:', score)
score = log_reg.score(X_test_features, y_test)
print('Train Accuracy Score:', score)

Train Accuracy Score: 0.383377308707124
Train Accuracy Score: 0.37618712627506157


In [39]:
feature_df = pd.read_csv('features.csv', skiprows=[1,2,3], low_memory=False)
feature_df.head()

,feature,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
0,2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
1,3,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
2,5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
3,10,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
4,20,-0.193837,-0.198527,0.201546,0.258556,0.775204,0.084794,-0.289294,-0.816410,0.043851,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


In [34]:
feature_df.sample(10).T

,4963,8557,10062,12155,9912,9990,4199,2665,2384,12135
feature,9515,14413,16619,19907,16403,16531,8375,4466,4117,19884
chroma_cens,-6.4009678364e-01,-1.8137915432e-01,-4.6748533845e-01,-5.5447924137e-01,-6.6032356024e-01,-5.1379144192e-01,1.8429424763e+00,-7.1316128969e-01,-2.5927108526e-01,1.0767548084e+00
chroma_cens.1,2.4348275661e+00,-7.6671309769e-02,2.1913425624e-01,-3.3329889178e-01,-5.6785285473e-01,6.0691136867e-02,2.2742567062e+00,-1.1531695127e+00,3.3509594202e-01,-1.7245726660e-02
chroma_cens.2,1.0266129673e-01,-4.6547159553e-01,-4.1683423519e-01,-2.4000993371e-01,-3.8555017114e-01,-1.5833829343e-01,3.2991211414e+00,-1.1898392439e+00,1.6926713288e-01,3.8349584793e-04
chroma_cens.3,1.0133403540e+00,-3.1954759359e-01,-1.6469989717e-01,-6.5702450275e-01,-4.2180828750e-02,-2.5575152040e-01,2.2097756863e+00,-1.2776225805e-01,2.1474672854e-01,8.1364291906e-01
chroma_cens.4,1.7008322477e+00,3.1073598191e-02,8.6848050356e-01,-5.7382559776e-01,-6.9000005722e-01,-5.4703372717e-01,2.0226800442e+00,-1.1846120358e+00,-1.6659145057e-01,-3.0765902996e-01
chroma_cens.5,-5.2355933189e-01,-3.9169836044e-01,-7.7050268650e-01,5.0970876217e-01,-8.2271009684e-01,-7.9649442434e-01,1.9766552448e+00,-4.9360311031e-01,-8.6366474628e-02,-2.4207864702e-01
chroma_cens.6,2.0253469050e-01,-1.1857179552e-01,-6.4850044250e-01,-4.6508347988e-01,-2.9511049390e-01,1.3021692038e+00,1.7524362803e+00,-8.6666250229e-01,5.5673253536e-01,2.8575667739e-01
chroma_cens.7,-5.9706294537e-01,-4.1815993190e-01,8.2416838408e-01,-1.1777867079e+00,8.9489680529e-01,-1.6382473707e-01,1.9974262714e+00,1.5187549591e+00,-3.7117877603e-01,1.1859983206e+00
chroma_cens.8,-3.9840289950e-01,3.4135006368e-02,9.0001180768e-02,-6.6112691164e-01,-5.7599797845e-02,-9.8842662573e-01,3.7246592045e+00,-6.5089988708e-01,-2.5799465179e-01,2.8068745136e-01


In [35]:
feature_df.shape

(12628, 519)